In [52]:
import pandas as pd
from statsmodels.tsa.api import VAR
from statsmodels.tools.eval_measures import rmse
from sklearn.model_selection import train_test_split
from statsmodels.tsa.base.datetools import dates_from_str


In [53]:
data = pd.read_csv("TimeSeriesData.csv")

In [54]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48 entries, 0 to 47
Data columns (total 20 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   Year                        48 non-null     int64  
 1   Quarter                     48 non-null     object 
 2   Month                       48 non-null     object 
 3   Day                         48 non-null     int64  
 4   stand_allocc                48 non-null     float64
 5   stand_allocc_TDC            48 non-null     float64
 6   stand_FI                    48 non-null     float64
 7   stand_LSM                   48 non-null     float64
 8   stand_SCIO                  48 non-null     float64
 9   stand_SLA                   48 non-null     float64
 10  stand_SPA                   48 non-null     float64
 11  stand_TDC                   48 non-null     float64
 12  unique_postings_allocc      48 non-null     int64  
 13  unique_postings_allocc_TDC  48 non-nu

In [55]:
data.head()

,Year,Quarter,Month,Day,stand_allocc,stand_allocc_TDC,stand_FI,stand_LSM,stand_SCIO,stand_SLA,stand_SPA,stand_TDC,unique_postings_allocc,unique_postings_allocc_TDC,unique_postings_FI,unique_postings_LSM,unique_postings_SCIO,unique_postings_SLA,unique_postings_SPA,unique_postings_TDC
0,2019,Qtr 1,January,31,117.11,117.11,114.92,116.43,105.43,105.02,109.86,104.06,2035114,2035114,4797,753,3384,2414,11348,6460
1,2019,Qtr 1,February,28,114.32,114.32,105.08,111.79,100.78,108.24,104.87,103.79,1986645,1986645,4386,723,3235,2488,10832,6443
2,2019,Qtr 1,March,31,114.85,114.85,119.64,113.65,105.43,117.51,114.37,107.77,1995804,1995804,4994,735,3384,2701,11814,6690
3,2019,Qtr 2,April,30,107.73,107.73,110.95,112.72,102.50,111.42,108.54,96.59,1872115,1872115,4631,729,3290,2561,11211,5996
4,2019,Qtr 2,May,31,101.01,101.01,110.97,111.02,103.77,111.24,108.80,98.73,1755404,1755404,4632,718,3331,2557,11238,6129


In [56]:
data.tail()

,Year,Quarter,Month,Day,stand_allocc,stand_allocc_TDC,stand_FI,stand_LSM,stand_SCIO,stand_SLA,stand_SPA,stand_TDC,unique_postings_allocc,unique_postings_allocc_TDC,unique_postings_FI,unique_postings_LSM,unique_postings_SCIO,unique_postings_SLA,unique_postings_SPA,unique_postings_TDC
43,2022,Qtr 3,August,31,151.20,151.20,106.01,85.20,151.25,158.18,130.38,120.65,2627607,2627607,4425,551,4855,3636,13467,7490
44,2022,Qtr 3,September,30,153.42,153.42,120.91,88.44,151.81,154.57,135.97,114.27,2666179,2666179,5047,572,4873,3553,14045,7094
45,2022,Qtr 4,October,31,154.23,154.23,90.92,91.07,141.47,140.65,117.70,98.10,2680293,2680293,3795,589,4541,3233,12158,6090
46,2022,Qtr 4,November,30,158.35,158.35,92.52,94.47,154.59,135.39,121.47,99.68,2751858,2751858,3862,611,4962,3112,12547,6188
47,2022,Qtr 4,December,31,150.14,150.14,94.61,81.02,162.38,131.08,122.93,100.92,2609199,2609199,3949,524,5212,3013,12698,6265


In [57]:
# Combine 'Month' and 'Year' columns to create a new 'Date' column
data['Date'] = pd.to_datetime(data['Year'].astype(str) + '-' + data['Month'].astype(str) + "-" +data['Day'].astype(str))
# Drop the original 'Month' and 'Year' columns if needed
data = data.drop(['Month', 'Year',"Quarter","Day"], axis=1)

# Set the 'Date' column as the index
data.set_index('Date', inplace=True)

In [58]:
data.sample(3)

,stand_allocc,stand_allocc_TDC,stand_FI,stand_LSM,stand_SCIO,stand_SLA,stand_SPA,stand_TDC,unique_postings_allocc,unique_postings_allocc_TDC,unique_postings_FI,unique_postings_LSM,unique_postings_SCIO,unique_postings_SLA,unique_postings_SPA,unique_postings_TDC
Date,,,,,,,,,,,,,,,,
2022-07-31,165.72,165.72,126.42,98.80,171.38,176.50,149.81,123.73,2879913,2879913,5277,639,5501,4057,15474,7681
2021-02-28,92.67,92.67,74.00,58.60,84.46,35.24,67.66,46.57,1610488,1610488,3089,379,2711,810,6989,2891
2019-05-31,101.01,101.01,110.97,111.02,103.77,111.24,108.80,98.73,1755404,1755404,4632,718,3331,2557,11238,6129


In [59]:
# Fit the VAR model
model = VAR(data)
model_fit = model.fit()

/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency M will be used.
  self._init_dates(dates, freq)


In [60]:
# Forecast the next 12 steps
forecast_steps = 12
forecast = model_fit.forecast(data.values, steps=forecast_steps)  # Use model_fit.forecast directly

In [61]:
# Convert the forecast results to a DataFrame
forecast_df = pd.DataFrame(forecast, index=pd.date_range(start=data.index[-1] + pd.DateOffset(1), periods=forecast_steps, freq='M'), columns=data.columns)

In [62]:
data = data.append(forecast_df)

<ipython-input-62-de6d6fac1f3e>:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data = data.append(forecast_df)


In [63]:
data.sample(5)

,stand_allocc,stand_allocc_TDC,stand_FI,stand_LSM,stand_SCIO,stand_SLA,stand_SPA,stand_TDC,unique_postings_allocc,unique_postings_allocc_TDC,unique_postings_FI,unique_postings_LSM,unique_postings_SCIO,unique_postings_SLA,unique_postings_SPA,unique_postings_TDC
2021-06-30,134.970000,134.970000,86.530000,81.640000,139.290000,107.070000,107.190000,97.460000,2.345458e+06,2.345458e+06,3612.000000,528.000000,4471.000000,2461.000000,11072.000000,6050.000000
2023-04-30,170.287492,170.287492,109.049232,92.513346,172.612116,153.915819,137.751131,118.723740,2.959288e+06,2.959288e+06,4551.873761,598.343055,5540.585618,3537.874953,14228.677385,7370.376390
2020-01-31,84.640000,84.640000,74.630000,80.250000,81.720000,78.050000,77.940000,69.140000,1.470824e+06,1.470824e+06,3115.000000,519.000000,2623.000000,1794.000000,8051.000000,4292.000000
2022-10-31,154.230000,154.230000,90.920000,91.070000,141.470000,140.650000,117.700000,98.100000,2.680293e+06,2.680293e+06,3795.000000,589.000000,4541.000000,3233.000000,12158.000000,6090.000000
2023-06-30,175.149273,175.149273,114.756521,95.452612,176.373456,162.889984,143.407289,122.102194,3.043781e+06,3.043781e+06,4790.094059,617.352247,5661.324501,3744.148102,14812.918907,7580.116312


In [64]:
data.to_csv("ForcastTimeSeries.csv")